# Calculate global overturning circulation

In [1]:
import cosima_cookbook as cc
from dask.distributed import Client

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cmocean as cm
import matplotlib.gridspec as gridspec
%matplotlib inline

# Stop annoying warnings coming out of xarray.
import warnings
warnings.filterwarnings('ignore')

In [2]:
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 7
Total threads: 28,Total memory: 128.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:44163,Workers: 7
Dashboard: /proxy/8787/status,Total threads: 28
Started: Just now,Total memory: 128.00 GiB
Comm: tcp://127.0.0.1:36535,Total threads: 4
Dashboard: /proxy/33189/status,Memory: 18.29 GiB
Nanny: tcp://127.0.0.1:33543,


In [3]:
# CM2 database
session_CM2 = cc.database.create_session('/g/data/p73/archive/non-CMIP/ACCESS-CM2/CM2.db')
expt_CM2_025 = 'cj877'
expt_CM2_1   = 'bz687'

In [4]:
# OM2 database
session_OM2  = cc.database.create_session('/g/data/ik11/databases/cosima_master.db')
expt_OM2_025 = '025deg_jra55_ryf9091_gadi'
expt_OM2_1   = '1deg_jra55_ryf9091_gadi'

In [5]:
path_to_data = '/g/data/x77/wgh581/Post_Process/ACCESS_CM2_025/'

If you get a 'read-only database' warning, it is because the database has not finished building. 

In [6]:
# Plotting info
ft_size = 16
plt.rcParams.update({'font.size': ft_size})
fig_path = '/g/data/x77/wgh581/Figures/Figures_ACCESS_CM2/'

### Calculate overturning

In [7]:
def compute_psi_rho(expt, session, frequency='1 monthly', nbound=None, start_time=None, end_time=None):
    rho = 1025 # mean density of sea-water in kg/m^3
    
    varlist = cc.querying.get_variables(session, expt)
    if varlist['name'].str.contains('ty_trans_rho_gm').any():
        GM = True
        print('GM is True')
        psiGM = cc.querying.getvar(expt, 'ty_trans_rho_gm', session, frequency=frequency, n=nbound, start_time=start_time, end_time=end_time)
        psiGM = psiGM.sum('grid_xt_ocean')
        psiGM = psiGM / (1e6*rho)
    else:
        GM = False
        print('GM is False')
        
    psi = cc.querying.getvar(expt, 'ty_trans_rho', session, frequency=frequency, n=nbound, start_time=start_time, end_time=end_time)      
    psi = psi / (1e6*rho) # converts kg/s to Sv
    psi = psi.sum('grid_xt_ocean').cumsum('potrho').mean(dim='time').load() 
    if GM:
        psi = psi + psiGM.mean('time')
        
    return psi.compute()

CM2-025

In [ ]:
year = np.arange(1, 500, 1)

for ii in year:
    
    if ii > 341:
        if len(str(int(ii))) == 1:
            year_tmp = '000' +str(ii)
        elif len(str(int(ii))) == 2:
            year_tmp = '00' + str(ii)
        elif len(str(int(ii))) == 3:
            year_tmp = '0' + str(ii)

        start_time = year_tmp + '-01-01'
        end_time   = year_tmp + '-12-31'
        data_name   = 'Overturning_' + year_tmp + '.nc'

        psi_tmp = compute_psi_rho(expt_CM2_025, session_CM2, start_time=start_time, end_time=end_time)
        psi_tmp.to_netcdf(path_to_data + 'overturning_tmp/' + data_name)
        print('Finished year ', year_tmp)

CM2-1

In [ ]:
year = np.arange(400, 500, 1)

for ii in year:
    
    year_tmp = '0' + str(ii)

    start_time = year_tmp + '-01-01'
    end_time   = year_tmp + '-12-31'
    data_name  = 'Overturning_CM2_1_' + year_tmp + '.nc'

    psi_tmp = compute_psi_rho(expt_CM2_1, session_CM2, start_time=start_time, end_time=end_time)
    psi_tmp.to_netcdf(path_to_data + '/Comparison_ACCESS_CM2_1/overturning_tmp/' + data_name)
    print('Finished year ', year_tmp)

OM2-025

In [ ]:
year = np.arange(2300, 2399, 1)

#for ii in year:
for ii in [2399]:    
    year_tmp = str(ii)

    start_time = year_tmp + '-01-01'
    end_time   = year_tmp + '-12-31'
    data_name  = 'Overturning_OM2_025_' + year_tmp + '.nc'

    psi_tmp = compute_psi_rho(expt_OM2_025, session_OM2, frequency='1 monthly', start_time=start_time, end_time=end_time)
    psi_tmp.to_netcdf(path_to_data + '/Comparison_ACCESS_OM2_025/overturning_tmp/' + data_name)
    print('Finished year ', year_tmp)

OM2-1

In [8]:
year = np.arange(2500, 2599, 1)

#for ii in year:
for ii in [2599]:
    
    year_tmp = str(ii)

    start_time = year_tmp + '-01-01'
    end_time   = year_tmp + '-12-31'
    data_name  = 'Overturning_OM2_1_' + year_tmp + '.nc'

    psi_tmp = compute_psi_rho(expt_OM2_1, session_OM2, frequency='1 monthly', start_time=start_time, end_time=end_time)
    psi_tmp.to_netcdf(path_to_data + '/Comparison_ACCESS_OM2_1/overturning_tmp/' + data_name)
    print('Finished year ', year_tmp)

GM is True


/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.04/lib/python3.9/site-packages/xarray/coding/times.py:673: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.04/lib/python3.9/site-packages/xarray/coding/times.py:673: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.04/lib/python3.9/site-packages/xarray/coding/times.py:673: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datet

Finished year  2599


### Load overturning data (individual files), concatenate and save output as one file

In [16]:
import glob
import xesmf
import os
import pandas as pd
import calendar
from tqdm import tqdm_notebook

In [21]:
def load_individual_files(file_name='overturning_tmp/*.nc'):
    dataarrays = []
    files = glob.glob(os.path.join(path_to_data, file_name))
    for f in tqdm_notebook(files, leave=False, desc='opening files'):
        dataarrays.append(xr.open_dataarray(f, decode_times=False))
    dataarray = xr.concat(dataarrays, dim='time', coords='all')
    if 'time' in dataarray.coords:
        time_units = dataarray.time.units
        decoded_time = xr.conventions.times.decode_cf_datetime(dataarray.time, time_units)
        dataarray.coords['time'] = ('time', decoded_time,
                                    {'long_name': 'time', 'decoded_using': time_units })
    return dataarray.load()

In [19]:
psi_CM2_025 = load_individual_files()

In [22]:
psi_CM2_1 = load_individual_files(file_name = 'Comparison_ACCESS_CM2_1/overturning_tmp/*.nc')

opening files:   0%|          | 0/100 [00:00<?, ?it/s]

In [25]:
psi_OM2_1 = load_individual_files(file_name = 'Comparison_ACCESS_OM2_1/overturning_tmp/*.nc')

opening files:   0%|          | 0/100 [00:00<?, ?it/s]

In [26]:
psi_OM2_025 = load_individual_files(file_name = 'Comparison_ACCESS_OM2_025/overturning_tmp/*.nc')

opening files:   0%|          | 0/100 [00:00<?, ?it/s]

In [27]:
psi_CM2_025.to_netcdf(path_to_data + 'psi_global_0_499.nc')

In [28]:
psi_CM2_1.to_netcdf(path_to_data + 'Comparison_ACCESS_CM2_1/psi_global_CM2_1_400_499.nc')

In [29]:
psi_OM2_1.to_netcdf(path_to_data + 'Comparison_ACCESS_OM2_1/psi_global_OM2_1_2500_2599.nc')

In [30]:
psi_OM2_025.to_netcdf(path_to_data + 'Comparison_ACCESS_OM2_025/psi_global_OM2_025_2300_2399.nc')

### Test if files work fine and then delete the infividual files/folders